# Fluffy recognition model

Trying to build a model that recognises whether an image has something fluffy in it.

## Prepare environment

Install & import fastai / fastbook libraries

In [ ]:
!pip install fastai==2.5
!pip install -Uqq fastbook

import fastbook
from fastbook import *
from fastai.vision.all import *

fastbook.setup_book()

     |████████████████████████████████| 188 kB 4.3 MB/s 
     |████████████████████████████████| 56 kB 5.3 MB/s 
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61
     |████████████████████████████████| 720 kB 4.1 MB/s 
     |████████████████████████████████| 1.2 MB 51.6 MB/s 
     |████████████████████████████████| 46 kB 4.9 MB/s 
     |████████████████████████████████| 51 kB 401 kB/s 
Mounted at /content/gdrive


Download images

In [ ]:
IMG_URL = "https://github.com/mihailthebuilder/fluffy-nb/raw/main/fluffy-images.tar.xz"
path = untar_data(IMG_URL)
get_image_files(path)

(#283) [Path('/root/.fastai/data/fluffy-images/132-bathrobe.jpg'),Path('/root/.fastai/data/fluffy-images/85-GEMSTONE.JPEG.jpeg.jpg'),Path('/root/.fastai/data/fluffy-images/uggfluffyeahpurple.jpg'),Path('/root/.fastai/data/fluffy-images/16-FISHING-HOOK.JPEG.jpeg.jpg'),Path('/root/.fastai/data/fluffy-images/tnbxayb.jpg'),Path('/root/.fastai/data/fluffy-images/7spfl3s.jpg'),Path('/root/.fastai/data/fluffy-images/116-towel.jpg'),Path('/root/.fastai/data/fluffy-images/27-NETTLE.JPEG.jpeg.jpg'),Path('/root/.fastai/data/fluffy-images/62-NAIL-TOOL.JPEG.jpeg.jpg'),Path('/root/.fastai/data/fluffy-images/59-PENCIL-SHARPENER.JPEG.jpeg.jpg')...]

## Prepare data

Prepare data for model training

In [ ]:
def is_fluffy(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_fluffy, item_tfms=Resize(500))

## Train model

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(2)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch,train_loss,valid_loss,error_rate,time
0,1.271651,0.741432,0.196429,00:05


epoch,train_loss,valid_loss,error_rate,time
0,0.958777,0.544320,0.250000,00:06
1,0.800037,0.567093,0.267857,00:06


The error rate for the model should be somewhere around 20-30%. This was the best accuracy I could achieve after tweaking...
- The number of images fed into the model (currently 283). I added images until the marginal improvement in model accuracy wasn't significant.
- The number of epochs while keeping an eye out for overfitting.
- The pixel sizing of the transformed images via `item_tfms`. Higher pixel sizings made the model require more GPU memory.
- Layers in the ResNet model architecture I was using. As with the epochs, you had to be careful about overfitting. Another issue was that using more layers required again more memory, and I often hit the limitation with my Colab Pro subscription.

## Try out model

Upload your image

In [ ]:
uploader = widgets.FileUpload()
uploader

FileUpload(value={}, description='Upload')

Apply model on image

In [ ]:
img = PILImage.create(uploader.data[0])
fluffy,_,probs = learn.predict(img)
print(f"Is this fluffy?: {fluffy}.")
print(f"Probability it's fluffy: {probs[1].item():.6f}")

Is this fluffy?: True.
Probability it's fluffy: 0.646041


## Todo
Try out EfficientNetV2 model architecture. [State of the art](https://paperswithcode.com/sota/image-classification-on-imagenet) and resource-light. [Link](https://github.com/rwightman/pytorch-image-models) to one source. 